In [20]:
import github3

In [22]:
import yaml

In [32]:
gh = github3.login(token='???')
org = gh.organization('dd-decaf')

In [61]:
MAPPING = {
    'ANNOTATIONS': 'gene-to-reactions',
    'GENOMICS': 'ice'
}
EXTRA = ['ILOOP_API']

def node_name(repo, key=''):
    repo = MAPPING.get(repo, repo)
    repo += ' ' + key
    repo = repo.replace('-', ' ')
    return ''.join(repo.title().split())

final = {}
for repo in org.iter_repos():
    compose = repo.contents('docker-compose.yml')
    if compose:
        yml = yaml.load(compose.decoded)
        if 'version' in yml:
            yml = yml['services']
        for k, v in yml.items():
            new_links = []
            for en in v.get('environment', []):
                e = en.split('=')[0]
                if e.endswith('_API') and e not in EXTRA:
                    new_links.append(node_name(e.replace('_API', '').replace('_', '-'), 'web'))
            if 'links' in v:
                v['links'] = [node_name(repo.name, s) for s in v['links']]
            else:
                v['links'] = []
            v['links'].extend(new_links)
            final[node_name(repo.name, k)] = v
with open('docker-compose.yml', 'w') as f:
    f.write(yaml.dump(final, default_flow_style=False))